In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import argparse
import logging

from mex.Detection_module import ObjectDetector
from mex.Cleaning_module import GalaxyCleaner
from mex.Petrosian_module import PetrosianCalculator
from mex.Segmentation_module import SegmentImage
from mex.Background_module import BackgroundEstimator
from mex.Utils_module import open_fits_image
from astroquery.vizier import Vizier
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from astroquery.sdss import SDSS
from astropy import coordinates as coords
from astropy import units as u
import hostphot
from matplotlib.patches import Ellipse

import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from astropy.visualization import make_lupton_rgb
from hostphot.cutouts import download_images
import numpy as np
import pandas as pd
import sep
import logging
import requests
from io import BytesIO
from astropy.table import Table
from astroquery.sdss import SDSS
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
from astropy import units as u
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
def arcsec_to_deg(sec):
    return sec / 3600.0

def arcmin_to_deg(arcmin):
    return arcmin / 60.0

def deg_to_arcmin(degree):
    return degree * 60.0



class Mask_stars:
        def __init__(self, ra, dec, size, path_in , name , path_out=None):
            self.ra = ra
            self.dec = dec
            self.size = size
            self.path_out = path_out
            self.path_in = path_in
            self.name = name
        @staticmethod   
        def query_legacy_survey_box( ra_min, ra_max, dec_min, dec_max,legacy_survey_dr):
            """Query Legacy Survey catalog for a rectangular region."""
            # Usa el DR especificado en __init__
            ls_api_url = f"https://www.legacysurvey.org/viewer/ls-{legacy_survey_dr}/cat.fits"
            params = {'ralo': ra_min, 'rahi': ra_max, 'declo': dec_min, 'dechi': dec_max}
            logging.info(f"Querying Legacy Survey {legacy_survey_dr} API: RA [{ra_min:.3f},{ra_max:.3f}], Dec [{dec_min:.3f},{dec_max:.3f}]")

            try:
                response = requests.get(ls_api_url, params=params, timeout=30) # Timeout de 30s
                response.raise_for_status() # Lanza error para 4xx/5xx

                if response.content:
                    # Leer la tabla FITS directamente desde la respuesta en memoria
                    with BytesIO(response.content) as f:
                        ls_table = Table.read(f, format='fits')
                    logging.info(f"Found {len(ls_table)} Legacy Survey sources in the region.")
                    # Asegurar columnas necesarias ('ra', 'dec', 'type')
                    if 'ra' not in ls_table.colnames or 'dec' not in ls_table.colnames or 'type' not in ls_table.colnames:
                        logging.error(f"Legacy Survey table from API missing required columns (ra, dec, type). Found: {ls_table.colnames}")
                        return None
                    
                    return ls_table
                else:
                    logging.warning("Legacy Survey API returned empty content.")
                    return None
            except requests.exceptions.RequestException as e:
                logging.error(f"Error querying Legacy Survey API: {e}")
                return None
            except Exception as e:
                logging.error(f"Error reading Legacy Survey FITS response: {e}")
                return None
            

        @staticmethod
        def gal_detected(ra, dec, size_arcmin=3):
            """Query SDSS for spectroscopically confirmed galaxies."""
            rad_deg = arcmin_to_deg(size_arcmin)
            # Ajuste para búsqueda rectangular (más eficiente para SQL)
            ra_min, ra_max = ra - rad_deg / np.cos(np.deg2rad(dec)), ra + rad_deg / np.cos(np.deg2rad(dec))
            dec_min, dec_max = dec - rad_deg, dec + rad_deg
            
            query = f"""
            SELECT
                p.objid, p.ra, p.dec, s.specobjid
            FROM PhotoObjAll AS p
            JOIN SpecObjAll AS s ON p.objid = s.bestobjid
            WHERE s.class = 'GALAXY'
                AND p.ra BETWEEN {ra_min} AND {ra_max}
                AND p.dec BETWEEN {dec_min} AND {dec_max}
            """
            logging.info(f"Querying SDSS for galaxies in RA [{ra_min:.3f},{ra_max:.3f}], Dec [{dec_min:.3f},{dec_max:.3f}]")
            try:
                results = SDSS.query_sql(query)
                print(results)
                if results:
                    logging.info(f"Found {len(results)} SDSS spec-confirmed galaxies.")
                    return results # Retorna tabla Astropy
                else:
                    logging.info("No SDSS spec-confirmed galaxies found.")
                    return None
            except Exception as e:
                logging.error(f"Error querying SDSS galaxies: {e}")
                return None
            

        @staticmethod
        def detect_stars(ra, dec, radius_arcmin=3):
            """Query Gaia catalog stars."""
            radius_deg = arcmin_to_deg(radius_arcmin) * u.deg
            center = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, frame='icrs')

            logging.info(f"Querying Gaia around ({ra:.4f}, {dec:.4f}) with radius {radius_arcmin:.2f} arcmin.")
            try:
                # Intentar con Gaia EDR3 o DR3 si está disponible, si no, DR2
                # v = Vizier(catalog="I/355/gaiadr3", columns=['RA_ICRS', 'DE_ICRS', 'Source', 'Mag_G']) # Gaia DR3
                v = Vizier(catalog="I/350/gaiaedr3", columns=['RA_ICRS', 'DE_ICRS', 'Source', 'phot_g_mean_mag']) # Gaia EDR3 con Mag_G
                # v = Vizier(catalog="I/345/gaia2", columns=['ra', 'dec', 'source_id','phot_g_mean_mag']) # Gaia DR2
                
                v.ROW_LIMIT = -1
                gaia_results = v.query_region(center, radius=radius_deg, catalog=["I/350/gaiaedr3"]) # O el catálogo que funcione
                print(gaia_results)
                if gaia_results:
                    gaia_table = gaia_results[0]
                    # Renombrar columnas para consistencia
                    if 'RA_ICRS' in gaia_table.colnames: gaia_table.rename_column('RA_ICRS', 'ra')
                    if 'DE_ICRS' in gaia_table.colnames: gaia_table.rename_column('DE_ICRS', 'dec')
                    #if 'Mag_G' in gaia_table.colnames: gaia_table.rename_column('Mag_G','phot_g_mean_mag') # phot_g_mean_mag es más estándar
                    # Asegurarse que las columnas necesarias existan
                    if 'ra' not in gaia_table.colnames or 'dec' not in gaia_table.colnames:
                        logging.error("Gaia table missing 'ra' or 'dec' columns.")
                        return None

                    logging.info(f"Found {len(gaia_table)} Gaia sources.")
                    return gaia_table
                else:
                    logging.warning("No Gaia sources found in the region.")
                    return None
            except Exception as e:
                logging.error(f"Error querying Vizier for Gaia: {e}")
                return None

        @staticmethod
        def gaia_dataset(ra_deg, dec_deg, size=2):

            try:
                coords = SkyCoord(ra=ra_deg*u.deg, dec=dec_deg*u.deg, frame='icrs')
                radius = u.Quantity(size, u.arcmin)
                query = f"""
                SELECT
                    source_id, ra, dec, phot_g_mean_mag,
                    classprob_dsc_combmod_galaxy,
                    classprob_dsc_combmod_quasar,
                    classprob_dsc_combmod_star,
                    ruwe
                FROM
                    gaiadr3.gaia_source
                WHERE
                    1=CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', {coords.ra.deg}, {coords.dec.deg}, {radius.to(u.deg).value}))
                """
                job = Gaia.launch_job_async(query)
                results = job.get_results()

                if not results:
                    return None
                clasificaciones = []
                prob_threshold = 0.5  

                for row in results:
                    prob_galaxy = row['classprob_dsc_combmod_galaxy'] if not np.ma.is_masked(row['classprob_dsc_combmod_galaxy']) else 0
                    prob_quasar = row['classprob_dsc_combmod_quasar'] if not np.ma.is_masked(row['classprob_dsc_combmod_quasar']) else 0
                    prob_star = row['classprob_dsc_combmod_star'] if not np.ma.is_masked(row['classprob_dsc_combmod_star']) else 0
                    max_prob = max(prob_galaxy, prob_quasar, prob_star)

                    if max_prob < prob_threshold:
                        clasificaciones.append("UKNW")
                    elif prob_galaxy == max_prob:
                        clasificaciones.append("GAL")
                    elif prob_quasar == max_prob:
                        clasificaciones.append("QSO")
                    elif prob_star == max_prob:
                        clasificaciones.append("STAR")
                    else:

                        clasificaciones.append("IND")


                results['Class'] = clasificaciones
                results['Class'].description = 'Classification with treshold > 0.5'
                print(clasificaciones)
                # Reordenar columnas para mejor visualización (opcional)
                column_order = ['source_id', 'ra', 'dec', 'phot_g_mean_mag', 'Clasificacion',
                                'classprob_dsc_combmod_star', 'classprob_dsc_combmod_galaxy',
                                'classprob_dsc_combmod_quasar', 'ruwe']
                results = results[column_order]

                return results

            except Exception as e:
                print(f"ERROR : {e}")
                return None



        def get_data(self):

            self.file_img = self.path_in +'/'+ self.name 
            survey = 'LegacySurvey'
            download_images(self.file_img, self.ra, self.dec, survey=survey,size=self.size)
            data_name = f"{self.file_img}/{survey}_g.fits"
            hdu_list = fits.open(data_name)
            data = fits.getdata(data_name)

            wcs = WCS(hdu_list[0].header)
            img_corners_pix = [(0, 0), (data.shape[1]-1, 0), (0, data.shape[0]-1), (data.shape[1]-1, data.shape[0]-1)]
            img_corners_world = wcs.pixel_to_world(*zip(*img_corners_pix))
            ra_min, ra_max = np.min(img_corners_world.ra.deg), np.max(img_corners_world.ra.deg)
            dec_min, dec_max = np.min(img_corners_world.dec.deg), np.max(img_corners_world.dec.deg)
            
            tabls =  self.__class__.query_legacy_survey_box(ra_min, ra_max, dec_min, dec_max,legacy_survey_dr='dr10')
            tabsdss =  self.__class__.gal_detected(self.ra, self.dec, size_arcmin=self.size)
            tabstars =  self.__class__.detect_stars(self.ra, self.dec, radius_arcmin=self.size)
            tabstars_2 = self.__class__.gaia_dataset( self.ra, self.dec, size=self.size)

            if tabsdss is None:
                print('No SDSS objects found')
                ra_sdss = None
                dec_sdss = None
            else:
                ra_sdss = tabsdss['ra'][(tabsdss['ra']<ra_max) & (tabsdss['ra']>ra_min)& (tabsdss['dec']<dec_max) & (tabsdss['dec']>dec_min)]
                dec_sdss = tabsdss['dec'][(tabsdss['ra']<ra_max) & (tabsdss['ra']>ra_min)& (tabsdss['dec']<dec_max) & (tabsdss['dec']>dec_min)]

            ra_stars = tabstars['ra'][(tabstars['ra']<ra_max) & (tabstars['ra']>ra_min)& (tabstars['dec']<dec_max) & (tabstars['dec']>dec_min)]
            dec_stars = tabstars['dec'][(tabstars['ra']<ra_max) & (tabstars['ra']>ra_min)& (tabstars['dec']<dec_max) & (tabstars['dec']>dec_min)]

            ra_stars_2 = tabstars_2['ra'][(tabstars_2['ra']<ra_max) & (tabstars_2['ra']>ra_min)& (tabstars_2['dec']<dec_max) & (tabstars_2['dec']>dec_min)]
            dec_stars_2 = tabstars_2['dec'][(tabstars_2['ra']<ra_max) & (tabstars_2['ra']>ra_min)& (tabstars_2['dec']<dec_max) & (tabstars_2['dec']>dec_min)]
            ra_dr3 = tabstars_2['ra'][(tabstars_2['ra']<ra_max) & (tabstars_2['ra']>ra_min)& (tabstars_2['dec']<dec_max) & (tabstars_2['dec']>dec_min)&(tabstars_2['Class']!='STAR')]
            dec_dr3 = tabstars_2['dec'][(tabstars_2['ra']<ra_max) & (tabstars_2['ra']>ra_min)& (tabstars_2['dec']<dec_max) & (tabstars_2['dec']>dec_min)&(tabstars_2['Class']!='STAR')]


            ra_psf = tabls[tabls['type']=='PSF']['ra']
            dec_psf = tabls[tabls['type']=='PSF']['dec']
            
            ra_dev = tabls[tabls['type']=='DEV']['ra']
            dec_dev = tabls[tabls['type']=='DEV']['dec']

            ra_ser = tabls[tabls['type']=='SER']['ra']
            dec_ser = tabls[tabls['type']=='SER']['dec']

            ra_rex = tabls[tabls['type']=='REX']['ra']
            dec_rex = tabls[tabls['type']=='REX']['dec']

            ra_duv = tabls[tabls['type']=='DUV']['ra']
            dec_duv = tabls[tabls['type']=='DUV']['dec']

            ra_exp = tabls[tabls['type']=='EXP']['ra']
            dec_exp = tabls[tabls['type']=='EXP']['dec']
            
            obj_x_psf, obj_y_psf = wcs.wcs_world2pix(ra_psf, dec_psf, 1)
            obj_x_exp, obj_y_exp = wcs.wcs_world2pix(ra_exp, dec_exp, 1)
            obj_x_dev, obj_y_dev = wcs.wcs_world2pix(ra_dev, dec_dev, 1)
            obj_x_ser, obj_y_ser = wcs.wcs_world2pix(ra_ser, dec_ser, 1)
            obj_x_rex, obj_y_rex = wcs.wcs_world2pix(ra_rex, dec_rex, 1)
            obj_x_duv, obj_y_duv = wcs.wcs_world2pix(ra_duv, dec_duv, 1)

            ob_x_stars, obj_y_stars = wcs.wcs_world2pix(ra_stars, dec_stars, 1)
            ob_x_stars_2, obj_y_stars_2 = wcs.wcs_world2pix(ra_stars_2, dec_stars_2, 1)
            ob_x_dr3, obj_y_dr3 = wcs.wcs_world2pix(ra_dr3, dec_dr3, 1)

            psf_tab = pd.DataFrame({'x':ra_psf,'y':dec_psf,'type':np.full(len(ra_psf),'PSF')})
            dev_tab = pd.DataFrame({'x':ra_dev,'y':dec_dev,'type':np.full(len(ra_dev),'DEV')})
            ser_tab = pd.DataFrame({'x':ra_ser,'y':dec_ser,'type':np.full(len(ra_ser),'SER')})
            rex_tab = pd.DataFrame({'x':ra_rex,'y':dec_rex,'type':np.full(len(ra_rex),'REX')})
            duv_tab = pd.DataFrame({'x':ra_duv,'y':dec_duv,'type':np.full(len(ra_duv),'DUV')})
            exp_tab = pd.DataFrame({'x':ra_exp,'y':dec_exp,'type':np.full(len(ra_exp),'EXP')})
            
            stars_tab = pd.DataFrame({'x':ra_stars,'y':dec_stars,'type':np.full(len(ra_stars),'STARS')})
            stars_tab_2 = pd.DataFrame({'x':ra_stars_2,'y':dec_stars_2,'type':np.full(len(ra_stars_2),'STARS_2')})
            dr3_tab = pd.DataFrame({'x':ra_dr3,'y':dec_dr3,'type':np.full(len(ra_dr3),'GAL_DR3')})
            if tabsdss is None:
                obj_x_sdss, obj_y_sdss = None, None
                #sdss_tab = pd.DataFrame({'x':ra_sdss,'y':dec_sdss,'type':np.full(len(ra_sdss),'SDSS')})
                tab_all = pd.concat([psf_tab,dev_tab,ser_tab, rex_tab,duv_tab,exp_tab,stars_tab,stars_tab_2,dr3_tab],ignore_index=True)
            else:
                obj_x_sdss, obj_y_sdss = wcs.wcs_world2pix(ra_sdss, dec_sdss, 1)
                sdss_tab = pd.DataFrame({'x':ra_sdss,'y':dec_sdss,'type':np.full(len(ra_sdss),'SDSS')})
                tab_all = pd.concat([psf_tab,dev_tab,ser_tab, rex_tab,duv_tab,exp_tab,sdss_tab,stars_tab,stars_tab_2,dr3_tab],ignore_index=True)

            return tab_all




        def make_mask(object, size=3, thresh_1= 1.5,minarea_1 = 5,deblend_nthresh_1 = 34, deblend_cont_1 = 0.005,
                    thresh_2= 1.5,minarea_2 = 10,deblend_nthresh_2 = 64, deblend_cont_2 = 0.1):

            obj = object
            path = '/home/polo/Escritorio/WORKSPACE/PhDTHESIS/DATA_ENVIRONMENT/f_chances_04.csv'

            ta_c = pd.read_csv(path)

            tab = ta_c[ta_c['dr8Id'] == obj ]
            ra = tab['RA_J2000'].values[0]
            dec = tab['Dec_J2000'].values[0]
            survey = 'LegacySurvey'
            size = size
            name = str(obj)
            file_img = '/home/polo/Escritorio/WORKSPACE/PhDTHESIS/ZOOBOT_IMGS/imgs_data/'+ name
            galaxy_name = str(obj)
            hostphot.cutouts.download_images(file_img, ra, dec, survey=survey,size=size)

            galaxy_image_r, header = open_fits_image(f'/home/polo/Escritorio/WORKSPACE/PhDTHESIS/ZOOBOT_IMGS/imgs_data/{str(obj)}/LegacySurvey_r.fits')
            galaxy_image_g, header = open_fits_image(f'/home/polo/Escritorio/WORKSPACE/PhDTHESIS/ZOOBOT_IMGS/imgs_data/{str(obj)}/LegacySurvey_g.fits')

            galaxy_image_z, header = open_fits_image(f'/home/polo/Escritorio/WORKSPACE/PhDTHESIS/ZOOBOT_IMGS/imgs_data/{str(obj)}/LegacySurvey_z.fits')

            tablas = self.get_data()

            galaxy_image = galaxy_image_r + galaxy_image_g + galaxy_image_z
            rgb_default = make_lupton_rgb(galaxy_image_z*0.6, galaxy_image_r*0.8, galaxy_image_g*1.2, Q=8.5, stretch=0.06)

            bkg_estimator = BackgroundEstimator(galaxy_name, galaxy_image)

            bkg_median_frame, bkg_std_frame, bkg_image_frame, galaxy_nobkg_frame = bkg_estimator.frame_background(image_fraction = 0.1, sigma_clipping = True, clipping_threshold = 3)

            # First detection
            detector = ObjectDetector(galaxy_name, galaxy_nobkg_frame)
            sep_catalog, sep_segmentation = detector.sep_detector(thresh = thresh_1, minarea = minarea_1, deblend_nthresh = deblend_nthresh_1, 
                                                            deblend_cont = deblend_cont_1, filter_type = 'matched',
                                                            bkg_std = bkg_std_frame, sub_bkg = False)
            cleaner = GalaxyCleaner(galaxy_nobkg_frame, sep_segmentation)
            x, y = len(sep_segmentation)//2, len(sep_segmentation[0])//2
            main_id = sep_segmentation[y,x]
            galaxy_clean_iso = cleaner.isophotes_filler(sep_catalog['theta'][main_id - 1])

            # Second detection
            detector2 = ObjectDetector(galaxy_name, galaxy_clean_iso)
            sep_catalog2, sep_segmentation2 = detector2.sep_detector(thresh = thresh_2, minarea = minarea_2, deblend_nthresh = deblend_nthresh_2, 
                                                            deblend_cont = deblend_cont_2, filter_type = 'matched',
                                                            bkg_std = bkg_std_frame, sub_bkg = False)
            cleaner2 = GalaxyCleaner(galaxy_clean_iso, sep_segmentation2)
            x2, y2 = len(sep_segmentation2)//2, len(sep_segmentation2[0])//2
            main_id2 = sep_segmentation2[y2,x2]
            galaxy_clean_iso2 = cleaner2.isophotes_filler(sep_catalog2['theta'][main_id2 - 1])
            galaxy_clean_flat = cleaner2.flat_filler(median = np.nan)

            x3, y3, a, b, theta, npix = sep_catalog.iloc[main_id - 1][['x', 'y', 'a', 'b', 'theta', 'npix']]
            xx, yy, aa, bb, thetat, npixx = sep_catalog['x', 'y', 'a', 'b', 'theta', 'npix']
            
            x_st, y_st = tablas['x'][tablas['type'].isin(['PSF', 'STARS', 'STARS_2'])].values, \
                        tablas['y'][tablas['type'].isin(['PSF', 'STARS', 'STARS_2'])].values
            x_gl , y_gl = tablas['x'][tablas['type'].isin(['SDSS', 'GAL_DR3'])].values, \
                    tablas['y'][tablas['type'].isin(['SDSS', 'GAL_DR3'])].values
            x_st = np.array(x_st)
            y_st = np.array(y_st)
            distancias = np.sqrt((x_st - xx)**2 + (y_st - yy)**2)

            indice_min = np.argmin(distancias)
            distancia_min = distancias[indice_min]
            radio = 10  # píxeles
            indices_cercanos = np.where(distancias < radio)[0]
            print(indices_cercanos)
            rp_calc = PetrosianCalculator(galaxy_clean_iso2, x3, y3, a, b, theta)
            eta, growth_curve, radius, rp, eta_flag = rp_calc.calculate_petrosian_radius(rp_thresh = 0.2, 
                                                                            aperture = "elliptical", 
                                                                            optimize_rp = True,
                                                                            interpolate_order = 3, 
                                                                            Naround = 3, rp_step = 0.05)
            
            
            segm = SegmentImage(galaxy_clean_iso, sep_segmentation, rp, x3, y3, a, b, theta)
            segmentation_original = segm._get_original()
            new_data = np.stack([galaxy_clean_flat.astype(bool)]*3, axis=-1)*rgb_default

            
            flux, area = segm._calculate_flux_and_area(x3, y3, a, b, theta, scale = 1 * rp)
            print(flux/area)
            mu_thresh = segm._average_intensity(k_segmentation = 1)
            print(mu_thresh)
            
            

            mask_bool = segmentation_original.astype(bool)
            mask_3d = np.stack([mask_bool]*3, axis=-1)
            masked_rgb = rgb_default * mask_3d
            new_data = np.stack([galaxy_clean_flat.astype(bool)]*3, axis=-1)*mask_3d
            
            plt.figure(figsize  = (12,12), dpi = 200)
            m, s = np.nanmedian(galaxy_image), np.nanstd(galaxy_image)
            plt.subplot(2,3,1)
            plt.title('Image', fontsize = 22)
            plt.imshow(rgb_default, origin = 'lower', vmin = m, vmax = m+(0.5*s), cmap = 'gray_r')
            plt.xticks(fontsize = 16)
            plt.yticks(fontsize = 16)
            plt.tick_params(direction = 'in', size = 7, left = True, right = True, bottom = True, top = True, 
                            color = 'k', width = 1)

            plt.subplot(2,3,2)
            plt.title('SEP', fontsize = 22)
            plt.imshow(galaxy_clean_iso2, origin = 'lower', vmin = m, vmax = m+(0.5*s), cmap = 'gray_r')
            plt.xticks(fontsize = 16)
            plt.yticks(fontsize = 16)
            plt.tick_params(direction = 'in', size = 7, left = True, right = True, bottom = True, top = True, 
                            color = 'k', width = 1)

            plt.subplot(2,3,3)
            plt.title('SEP seg', fontsize = 22)
            plt.imshow(segmentation_original, origin = 'lower', cmap = 'viridis')
            plt.xticks(fontsize = 16)
            plt.yticks(fontsize = 16)
            plt.tick_params(direction = 'in', size = 7, left = True, right = True, bottom = True, top = True, 
                            color = 'k', width = 1)

            plt.subplot(2,3,4)
            plt.title('Intensity Limited', fontsize = 22)
            plt.imshow(masked_rgb, origin = 'lower', vmax = 1, vmin = 0, cmap = 'gray_r')
            plt.xticks(fontsize = 16)
            plt.yticks(fontsize = 16)
            plt.tick_params(direction = 'in', size = 7, left = True, right = True, bottom = True, top = True, 
                            color = 'k', width = 1)


            ax = plt.subplot(2,3,6)
            plt.title('SEP 2', fontsize = 22)
            m, s = np.nanmedian(galaxy_nobkg_frame), np.nanstd(galaxy_nobkg_frame)
            plt.imshow(galaxy_nobkg_frame, origin = 'lower', vmin = m, vmax = m+(3*s), cmap = 'gray_r')
            for i in range(len(sep_catalog2)):
                e = Ellipse(xy=(sep_catalog2['x'][i], sep_catalog2['y'][i]),
                            width=6*sep_catalog2['a'][i],
                            height=6*sep_catalog2['b'][i],
                            angle=sep_catalog2['theta'][i] * 180. / np.pi)
                e.set_facecolor('none')
                e.set_edgecolor('red')
                ax.add_artist(e)



            plt.xticks(fontsize = 16)
            plt.yticks(fontsize = 16)
            plt.tick_params(direction = 'in', size = 7, left = True, right = True, bottom = True, top = True, 
                            color = 'k', width = 1)
            
            ax2 = plt.subplot(2,3,5)
            plt.title('SEP 1', fontsize = 22)
            m, s = np.nanmedian(galaxy_nobkg_frame), np.nanstd(galaxy_nobkg_frame)
            plt.imshow(galaxy_nobkg_frame, origin = 'lower', vmin = m, vmax = m+(3*s), cmap = 'gray_r')
            for i in range(len(sep_catalog)):
                e = Ellipse(xy=(sep_catalog['x'][i], sep_catalog['y'][i]),
                            width=6*sep_catalog['a'][i],
                            height=6*sep_catalog['b'][i],
                            angle=sep_catalog['theta'][i] * 180. / np.pi)
                e.set_facecolor('none')
                e.set_edgecolor('red')
                ax2.add_artist(e)



            plt.xticks(fontsize = 16)
            plt.yticks(fontsize = 16)
            plt.tick_params(direction = 'in', size = 7, left = True, right = True, bottom = True, top = True, 
                            color = 'k', width = 1)
            plt.savefig(f'/home/polo/Escritorio/WORKSPACE/PhDTHESIS/DATA_ENVIRONMENT/output_images/{str(obj)}.jpg', bbox_inches = 'tight')
            #plt.savefig('output_images/segmentation_comparison.jpg', bbox_inches = 'tight')
        
